In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf

In [2]:
# Importing all the libraries : 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

In [3]:
df_train=pd.read_csv("train.tsv",sep="\t")
df_test=pd.read_csv("validation.tsv",sep="\t")

In [4]:
df_train['Tweet'] = df_train['Tweet'].fillna('Tweet was useless')
df_test['Tweet'] = df_test['Tweet'].fillna('Tweet was useless')

In [5]:
import nltk
nltk.download('stopwords')
stop_word = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\arjun
[nltk_data]     vyshakh.s\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def clean(text):

    #     remove urls
    text = re.sub(r'http\S+', " ", text)

    #     remove mentions
    text = re.sub(r'@\w+',' ',text)

    #     remove hastags
    text = re.sub(r'#\w+', ' ', text)

    #     remove digits
    text = re.sub(r'\d+', ' ', text)

    #     remove html tags
    text = re.sub('r<.*?>',' ', text)
    
    #     remove stop words 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_word])
    
      
    return text

In [7]:
df_train['Tweet'] = df_train['Tweet'].apply(lambda x: clean(x))

In [8]:
df_test['Tweet'] = df_test['Tweet'].apply(lambda x: clean(x))

In [9]:
df_train

,id,Tweet,Claim,Stance,Premise
0,3054935,"It's dangerous hold RNC, let's send students t...",school closures,FAVOR,1
1,453444,If masks important public made aware ONLY mask...,face masks,AGAINST,1
2,211600,But schools close gonna occupy graves? Cause p...,school closures,FAVOR,1
3,3049718,Governor I thought order another mandate. Clea...,stay at home orders,FAVOR,1
4,5039679580998,. You reporting % uptake masks many EU countri...,face masks,FAVOR,1
...,...,...,...,...,...
3551,333323,"Enjoy peace &amp; quiet, nature , slower pace ...",stay at home orders,FAVOR,1
3552,910360,The next fucking asshole tells data kids trans...,school closures,FAVOR,1
3553,769878,Learning constant...then . Why covid situation...,school closures,FAVOR,1
3554,648731,Every lockdown One Common Clause “Essential Se...,school closures,AGAINST,1


In [10]:
df_test['Claim'].unique()

array(['face masks', 'stay at home orders', 'school closures',
       "My new mask is sooo cute I don't wanna take it off even at home and the best part is its sustainable    face masks"],
      dtype=object)

In [11]:
df_train['Stance'].unique()

array(['FAVOR', 'AGAINST', 'NONE'], dtype=object)

In [12]:
l = { 'school closures':0, 'face masks':1, 'stay at home orders':2, "My new mask is sooo cute I don't wanna take it off even at home and the best part is its sustainable    face masks":3}
m = { 'FAVOR':0, 'AGAINST':1, 'NONE':2 }

In [13]:
df_train['Claim'] = df_train['Claim'].map(l)
df_test['Claim']  = df_test['Claim'].map(l)

In [14]:
df_train['Stance'] = df_train['Stance'].map(m)
df_test['Stance']  = df_test['Stance'].map(m)

In [15]:
df_train

,id,Tweet,Claim,Stance,Premise
0,3054935,"It's dangerous hold RNC, let's send students t...",0,0,1
1,453444,If masks important public made aware ONLY mask...,1,1,1
2,211600,But schools close gonna occupy graves? Cause p...,0,0,1
3,3049718,Governor I thought order another mandate. Clea...,2,0,1
4,5039679580998,. You reporting % uptake masks many EU countri...,1,0,1
...,...,...,...,...,...
3551,333323,"Enjoy peace &amp; quiet, nature , slower pace ...",2,0,1
3552,910360,The next fucking asshole tells data kids trans...,0,0,1
3553,769878,Learning constant...then . Why covid situation...,0,0,1
3554,648731,Every lockdown One Common Clause “Essential Se...,0,1,1


In [16]:
df1 = df_train[df_train['Tweet'].str.contains('\?')]
df1

,id,Tweet,Claim,Stance,Premise
1,453444,If masks important public made aware ONLY mask...,1,1,1
2,211600,But schools close gonna occupy graves? Cause p...,0,0,1
4,5039679580998,. You reporting % uptake masks many EU countri...,1,0,1
10,804806,you’re telling student teacher get coronavirus...,0,0,1
14,3538014,Option homeschooling rich. Not poor. You gave ...,0,0,1
...,...,...,...,...,...
3540,440294,Are listening experts yet? Or economy still th...,2,0,0
3541,893072,If masks work care I one?:thinking_face: These...,1,1,1
3553,769878,Learning constant...then . Why covid situation...,0,0,1
3554,648731,Every lockdown One Common Clause “Essential Se...,0,1,1


In [17]:
df2 = df_test[df_test['Tweet'].str.contains('\?')]
df2

,id,Tweet,Claim,Stance,Premise
1,2367433035,Who ready ?,2,2,0
14,676317,"""Often fatal""?? Are really dumb? There . % sur...",1,1,1
21,441615,How many would’ve agreed known would + months!...,2,1,1
32,648996,Wasn’t plan digital India? So now? Let’s suppo...,0,0,0
34,3702424,"Ok, can't trust educators safe distance wear m...",0,1,1
...,...,...,...,...,...
585,3686703949,How many children died ?,0,2,0
588,2224503,Your toilet cleaner washed water chemical! Tha...,1,1,1
593,904598,Why can’t people take virus seriously wear dam...,1,0,0
597,2316008,"If retirement age, *much* higher chance dying ...",2,1,1


In [18]:
li1 = df1.index.tolist() 
li2 = df2.index.tolist()

In [19]:
df_train['contains ?']=0

for i in range(len(li1)):
    df_train['contains ?'].iloc[li1[i]]=1

df_train

,id,Tweet,Claim,Stance,Premise,contains ?
0,3054935,"It's dangerous hold RNC, let's send students t...",0,0,1,0
1,453444,If masks important public made aware ONLY mask...,1,1,1,1
2,211600,But schools close gonna occupy graves? Cause p...,0,0,1,1
3,3049718,Governor I thought order another mandate. Clea...,2,0,1,0
4,5039679580998,. You reporting % uptake masks many EU countri...,1,0,1,1
...,...,...,...,...,...,...
3551,333323,"Enjoy peace &amp; quiet, nature , slower pace ...",2,0,1,0
3552,910360,The next fucking asshole tells data kids trans...,0,0,1,0
3553,769878,Learning constant...then . Why covid situation...,0,0,1,1
3554,648731,Every lockdown One Common Clause “Essential Se...,0,1,1,1


In [20]:
df_test['contains ?']=0

for i in range(len(li2)):
    df_test['contains ?'].iloc[li2[i]]=1

df_test

,id,Tweet,Claim,Stance,Premise,contains ?
0,7212131785749,Ordered mask cute chain attached don’t lose da...,1,0,0,0
1,2367433035,Who ready ?,2,2,0,1
2,496011,", families know it. Another , + hospitalized. ...",1,0,1,0
3,1490686,When ask I life I say I wore fucking mask.,1,0,0,0
4,1209366,Taylor putting album physical stores definitio...,2,0,0,0
...,...,...,...,...,...,...
595,2949472,one actually protect :clapping_hands_medium-li...,1,1,1,0
596,208269,I tested positive Covid. Got friend. I assumed...,1,0,1,0
597,2316008,"If retirement age, *much* higher chance dying ...",2,1,1,1
598,659269,"Houston really problem moment, considering May...",1,0,0,0


In [21]:
tokenized_tweet = df_train['Tweet'].apply(lambda x: x.split())
tokenized_tweet

0       [It's, dangerous, hold, RNC,, let's, send, stu...
1       [If, masks, important, public, made, aware, ON...
2       [But, schools, close, gonna, occupy, graves?, ...
3       [Governor, I, thought, order, another, mandate...
4       [., You, reporting, %, uptake, masks, many, EU...
                              ...                        
3551    [Enjoy, peace, &amp;, quiet,, nature, ,, slowe...
3552    [The, next, fucking, asshole, tells, data, kid...
3553    [Learning, constant...then, ., Why, covid, sit...
3554    [Every, lockdown, One, Common, Clause, “Essent...
3555    [Don’t, teachers, learners, fall, banner, “all...
Name: Tweet, Length: 3556, dtype: object

In [22]:
tokenized_tweet1 = df_test['Tweet'].apply(lambda x: x.split())
tokenized_tweet1

0      [Ordered, mask, cute, chain, attached, don’t, ...
1                                        [Who, ready, ?]
2      [,, families, know, it., Another, ,, +, hospit...
3      [When, ask, I, life, I, say, I, wore, fucking,...
4      [Taylor, putting, album, physical, stores, def...
                             ...                        
595    [one, actually, protect, :clapping_hands_mediu...
596    [I, tested, positive, Covid., Got, friend., I,...
597    [If, retirement, age,, *much*, higher, chance,...
598    [Houston, really, problem, moment,, considerin...
599    [Obviously, I, simply, practicing, stand-up, a...
Name: Tweet, Length: 600, dtype: object

In [23]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
tokenized_tweet.head()

0    [it', danger, hold, rnc,, let', send, student,...
1    [if, mask, import, public, made, awar, onli, m...
2    [but, school, close, gonna, occupi, graves?, c...
3    [governor, i, thought, order, anoth, mandate.,...
4    [., you, report, %, uptak, mask, mani, eu, cou...
Name: Tweet, dtype: object

In [24]:
# combine words into single sentence
for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
df_train['Tweet'] = tokenized_tweet
df_train.head()

,id,Tweet,Claim,Stance,Premise,contains ?
0,3054935,"it' danger hold rnc, let' send student teacher...",0,0,1,0
1,453444,if mask import public made awar onli mask prot...,1,1,1,1
2,211600,but school close gonna occupi graves? caus pol...,0,0,1,1
3,3049718,governor i thought order anoth mandate. clearl...,2,0,1,0
4,5039679580998,. you report % uptak mask mani eu countri % br...,1,0,1,1


In [25]:
tokenized_tweet1 = tokenized_tweet1.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
tokenized_tweet1.head()

0    [order, mask, cute, chain, attach, don’t, lose...
1                                      [who, readi, ?]
2    [,, famili, know, it., anoth, ,, +, hospitaliz...
3    [when, ask, i, life, i, say, i, wore, fuck, ma...
4    [taylor, put, album, physic, store, definit, c...
Name: Tweet, dtype: object

In [26]:
for i in range(len(tokenized_tweet1)):
        tokenized_tweet1[i] = " ".join(tokenized_tweet1[i])
    
df_test['Tweet'] = tokenized_tweet1
df_test.head()

,id,Tweet,Claim,Stance,Premise,contains ?
0,7212131785749,order mask cute chain attach don’t lose day :r...,1,0,0,0
1,2367433035,who readi ?,2,2,0,1
2,496011,", famili know it. anoth , + hospitalized. i su...",1,0,1,0
3,1490686,when ask i life i say i wore fuck mask.,1,0,0,0
4,1209366,taylor put album physic store definit carri pe...,2,0,0,0


In [27]:
list_1 = list(df_train['Tweet'])
list_2 = list(df_test['Tweet'])

In [28]:
list_1 = list_1 + list_2
list_1

["it' danger hold rnc, let' send student teacher back school",
 'if mask import public made awar onli mask protect wearer covid - n mask?',
 'but school close gonna occupi graves? caus politicians(protected) smoker (protected) drunkards(protected) school kid &amp; teacher veri exposed.',
 'governor i thought order anoth mandate. clearli know numbers, &amp; save - children &amp; mine!',
 '. you report % uptak mask mani eu countri % brit stay home afraid go out. could reason lack mask wear itself? i know i go out!',
 'the strongest motiv hongkongers’ distrust china.',
 'hehe.... what u bro :star-struck::hearts:',
 'littl wet men minc along perfect analog mask gimp.',
 'it arrog school an hospitals. i total agre incompet state govt. societi instead fight privat sector must go fight govern ask provid best public educ health.',
 'their emot toll children',
 'you’r tell student teacher get coronaviru school close hour everyon continu go school that? dumbass came idea...?',
 "over , american 

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english',ngram_range=(1,2))
# tokenize and build vocab
tfidfvectorizer.fit(list_1)

TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

In [30]:
tfidf_tokens = tfidfvectorizer.get_feature_names()
len(tfidf_tokens)

42734

In [31]:
list_1 = list(df_train['Tweet'])

In [32]:
idex = df_train['id'].tolist()

In [33]:
tfidf_wm = tfidfvectorizer.transform(list_1)
df_tfidfvect_train = pd.DataFrame(data = tfidf_wm.toarray(),index = idex,columns = tfidf_tokens)
df_tfidfvect_train.head()

,_______,_______ social,_hearts,_hearts innocent,_hearts smiling_face_with_,aanouc,aanouc visit,aaps,aaps suggest,ab,...,zoomsgiv christzoom,zoomukkah,zoomukkah kwanzoom,आइज,आइज bat,गर,पछ,पछ strike,सक,सक पछ
3054935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
453444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3049718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5039679580998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_train = df_train.set_index('id')

In [35]:
df_tfidfvect_train['Claim'] = df_train['Claim']
df_tfidfvect_train

,_______,_______ social,_hearts,_hearts innocent,_hearts smiling_face_with_,aanouc,aanouc visit,aaps,aaps suggest,ab,...,zoomukkah,zoomukkah kwanzoom,आइज,आइज bat,गर,पछ,पछ strike,सक,सक पछ,Claim
3054935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
453444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
211600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3049718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5039679580998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
910360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
769878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
648731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [36]:
idex = df_test['id'].tolist()

In [37]:
tfidf_wm = tfidfvectorizer.transform(list_2)
df_tfidfvect_test = pd.DataFrame(data = tfidf_wm.toarray(),index = idex,columns = tfidf_tokens)
df_tfidfvect_test.head()

,_______,_______ social,_hearts,_hearts innocent,_hearts smiling_face_with_,aanouc,aanouc visit,aaps,aaps suggest,ab,...,zoomsgiv christzoom,zoomukkah,zoomukkah kwanzoom,आइज,आइज bat,गर,पछ,पछ strike,सक,सक पछ
7212131785749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2367433035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1490686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1209366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df_test = df_test.set_index('id')

In [39]:
df_tfidfvect_test['Claim'] = df_test['Claim']
df_tfidfvect_test

,_______,_______ social,_hearts,_hearts innocent,_hearts smiling_face_with_,aanouc,aanouc visit,aaps,aaps suggest,ab,...,zoomukkah,zoomukkah kwanzoom,आइज,आइज bat,गर,पछ,पछ strike,सक,सक पछ,Claim
7212131785749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2367433035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
496011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1490686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1209366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2949472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
208269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2316008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
659269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [40]:
train_Stance = df_train['Stance']
test_Stance = df_test['Stance']
train_Premise = df_train['Premise']
test_Premise = df_test['Premise']

In [41]:
df_tfidfvect_test['contains ?'] = df_test['contains ?']
df_tfidfvect_train['contains ?'] = df_train['contains ?']
df_tfidfvect_train

,_______,_______ social,_hearts,_hearts innocent,_hearts smiling_face_with_,aanouc,aanouc visit,aaps,aaps suggest,ab,...,zoomukkah kwanzoom,आइज,आइज bat,गर,पछ,पछ strike,सक,सक पछ,Claim,contains ?
3054935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
453444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
211600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3049718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
5039679580998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
910360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
769878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
648731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


## Preparing the Model

In [42]:
#from sklearn import cross_validation
from sklearn.model_selection import train_test_split
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train, features_test, labels_train, labels_test = train_test_split(df_tfidfvect_train, train_Stance, test_size=0.3, random_state=1)

In [44]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import decomposition
from sklearn.model_selection import GridSearchCV

# Grid Search - Used to find best combination of parameters
XGB_model = xgb.XGBClassifier(objective='multi:softmax', colsample_bytree=1, seed=0, learning_rate=0.3, max_depth=4, n_estimators=100, n_jobs=12)
 

#model.fit(X, y)
XGB_model.fit(features_train, labels_train)
prediction = XGB_model.predict(features_test)
from sklearn.metrics import accuracy_score
print (accuracy_score(prediction, labels_test))

0.6682286785379569


In [49]:
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-6, random_state=0)
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
print (accuracy_score(prediction, labels_test))

0.619493908153702


In [45]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=SVC(),n_estimators=10, random_state=0).fit(features_train, labels_train)
prediction=clf.predict(features_test)
print (accuracy_score(prediction, labels_test))

KeyboardInterrupt: ignored

In [ ]:
prediction=clf.predict(features_test)
print (accuracy_score(prediction, labels_test))

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[100], 'max_depth':[3], 'learning_rate':[0.1,0.3]}
dt = GradientBoostingClassifier(random_state=0)
clf = GridSearchCV(dt, parameters)


clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
print(accuracy_score(prediction, labels_test))

0.647610121836926


In [43]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
 
clf = CatBoostClassifier(iterations=1000, learning_rate=0.1)
clf.fit(features_train, labels_train, 
        eval_set=(features_test, labels_test), 
        verbose=False
)
 
prediction=clf.predict(features_test)
print (accuracy_score(prediction, labels_test))
 

0.6663542642924086


In [45]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
estimators = [
     ('catboost', clf),
     ('XGBoost', XGB_model)
]
lgclassifier = LogisticRegression(random_state=0)
sclf = StackingClassifier(estimators=estimators,final_estimator=lgclassifier,cv=10)
sclf.fit(features_train, labels_train)
prediction = sclf.predict(features_test)
print(accuracy_score(prediction, labels_test))

0:	learn: 1.0714499	total: 135ms	remaining: 2m 15s
1:	learn: 1.0476183	total: 286ms	remaining: 2m 22s
2:	learn: 1.0309258	total: 432ms	remaining: 2m 23s
3:	learn: 1.0150183	total: 561ms	remaining: 2m 19s
4:	learn: 0.9996126	total: 698ms	remaining: 2m 18s
5:	learn: 0.9878205	total: 832ms	remaining: 2m 17s
6:	learn: 0.9791843	total: 978ms	remaining: 2m 18s
7:	learn: 0.9728351	total: 1.11s	remaining: 2m 17s
8:	learn: 0.9657334	total: 1.25s	remaining: 2m 17s
9:	learn: 0.9606416	total: 1.38s	remaining: 2m 16s
10:	learn: 0.9525555	total: 1.52s	remaining: 2m 16s
11:	learn: 0.9445354	total: 1.66s	remaining: 2m 16s
12:	learn: 0.9361199	total: 1.8s	remaining: 2m 16s
13:	learn: 0.9318000	total: 1.93s	remaining: 2m 15s
14:	learn: 0.9260376	total: 2.07s	remaining: 2m 16s
15:	learn: 0.9221229	total: 2.2s	remaining: 2m 15s
16:	learn: 0.9180523	total: 2.34s	remaining: 2m 15s
17:	learn: 0.9152151	total: 2.47s	remaining: 2m 14s
18:	learn: 0.9105569	total: 2.6s	remaining: 2m 14s
19:	learn: 0.9084039	tota

160:	learn: 0.7479610	total: 22.3s	remaining: 1m 55s
161:	learn: 0.7477024	total: 22.4s	remaining: 1m 55s
162:	learn: 0.7472141	total: 22.5s	remaining: 1m 55s
163:	learn: 0.7462724	total: 22.7s	remaining: 1m 55s
164:	learn: 0.7451703	total: 22.8s	remaining: 1m 55s
165:	learn: 0.7448285	total: 22.9s	remaining: 1m 55s
166:	learn: 0.7441831	total: 23.1s	remaining: 1m 55s
167:	learn: 0.7432144	total: 23.2s	remaining: 1m 54s
168:	learn: 0.7417973	total: 23.4s	remaining: 1m 54s
169:	learn: 0.7412232	total: 23.5s	remaining: 1m 54s
170:	learn: 0.7406870	total: 23.6s	remaining: 1m 54s
171:	learn: 0.7396632	total: 23.8s	remaining: 1m 54s
172:	learn: 0.7392564	total: 23.9s	remaining: 1m 54s
173:	learn: 0.7388756	total: 24s	remaining: 1m 54s
174:	learn: 0.7384412	total: 24.2s	remaining: 1m 53s
175:	learn: 0.7378962	total: 24.3s	remaining: 1m 53s
176:	learn: 0.7373233	total: 24.4s	remaining: 1m 53s
177:	learn: 0.7363276	total: 24.6s	remaining: 1m 53s
178:	learn: 0.7358379	total: 24.7s	remaining: 1m

316:	learn: 0.6677363	total: 43.3s	remaining: 1m 33s
317:	learn: 0.6674245	total: 43.5s	remaining: 1m 33s
318:	learn: 0.6672369	total: 43.6s	remaining: 1m 33s
319:	learn: 0.6669273	total: 43.7s	remaining: 1m 32s
320:	learn: 0.6666393	total: 43.9s	remaining: 1m 32s
321:	learn: 0.6660950	total: 44s	remaining: 1m 32s
322:	learn: 0.6657765	total: 44.1s	remaining: 1m 32s
323:	learn: 0.6653059	total: 44.3s	remaining: 1m 32s
324:	learn: 0.6648797	total: 44.4s	remaining: 1m 32s
325:	learn: 0.6645491	total: 44.5s	remaining: 1m 32s
326:	learn: 0.6638085	total: 44.7s	remaining: 1m 31s
327:	learn: 0.6635688	total: 44.8s	remaining: 1m 31s
328:	learn: 0.6632401	total: 44.9s	remaining: 1m 31s
329:	learn: 0.6629182	total: 45.1s	remaining: 1m 31s
330:	learn: 0.6626730	total: 45.2s	remaining: 1m 31s
331:	learn: 0.6622853	total: 45.3s	remaining: 1m 31s
332:	learn: 0.6618466	total: 45.5s	remaining: 1m 31s
333:	learn: 0.6613884	total: 45.6s	remaining: 1m 30s
334:	learn: 0.6610290	total: 45.7s	remaining: 1m

472:	learn: 0.6179498	total: 1m 4s	remaining: 1m 11s
473:	learn: 0.6176789	total: 1m 4s	remaining: 1m 11s
474:	learn: 0.6174049	total: 1m 4s	remaining: 1m 11s
475:	learn: 0.6172067	total: 1m 4s	remaining: 1m 11s
476:	learn: 0.6170558	total: 1m 4s	remaining: 1m 11s
477:	learn: 0.6168794	total: 1m 5s	remaining: 1m 11s
478:	learn: 0.6165090	total: 1m 5s	remaining: 1m 10s
479:	learn: 0.6161719	total: 1m 5s	remaining: 1m 10s
480:	learn: 0.6158687	total: 1m 5s	remaining: 1m 10s
481:	learn: 0.6153429	total: 1m 5s	remaining: 1m 10s
482:	learn: 0.6151504	total: 1m 5s	remaining: 1m 10s
483:	learn: 0.6149672	total: 1m 5s	remaining: 1m 10s
484:	learn: 0.6148127	total: 1m 6s	remaining: 1m 10s
485:	learn: 0.6146287	total: 1m 6s	remaining: 1m 9s
486:	learn: 0.6144167	total: 1m 6s	remaining: 1m 9s
487:	learn: 0.6141441	total: 1m 6s	remaining: 1m 9s
488:	learn: 0.6137197	total: 1m 6s	remaining: 1m 9s
489:	learn: 0.6134201	total: 1m 6s	remaining: 1m 9s
490:	learn: 0.6132633	total: 1m 6s	remaining: 1m 9s

628:	learn: 0.5803855	total: 1m 25s	remaining: 50.6s
629:	learn: 0.5801313	total: 1m 25s	remaining: 50.4s
630:	learn: 0.5794138	total: 1m 25s	remaining: 50.3s
631:	learn: 0.5792290	total: 1m 26s	remaining: 50.1s
632:	learn: 0.5790330	total: 1m 26s	remaining: 50s
633:	learn: 0.5788426	total: 1m 26s	remaining: 49.9s
634:	learn: 0.5786289	total: 1m 26s	remaining: 49.7s
635:	learn: 0.5785035	total: 1m 26s	remaining: 49.6s
636:	learn: 0.5781549	total: 1m 26s	remaining: 49.5s
637:	learn: 0.5779917	total: 1m 26s	remaining: 49.3s
638:	learn: 0.5777566	total: 1m 27s	remaining: 49.2s
639:	learn: 0.5776174	total: 1m 27s	remaining: 49s
640:	learn: 0.5773570	total: 1m 27s	remaining: 48.9s
641:	learn: 0.5772035	total: 1m 27s	remaining: 48.8s
642:	learn: 0.5770335	total: 1m 27s	remaining: 48.7s
643:	learn: 0.5768368	total: 1m 27s	remaining: 48.5s
644:	learn: 0.5765155	total: 1m 27s	remaining: 48.4s
645:	learn: 0.5763196	total: 1m 28s	remaining: 48.2s
646:	learn: 0.5761060	total: 1m 28s	remaining: 48.

...

380:	learn: 0.6376156	total: 49.2s	remaining: 1m 19s
381:	learn: 0.6372882	total: 49.3s	remaining: 1m 19s
382:	learn: 0.6369764	total: 49.5s	remaining: 1m 19s
383:	learn: 0.6360077	total: 49.6s	remaining: 1m 19s
384:	learn: 0.6358237	total: 49.7s	remaining: 1m 19s
385:	learn: 0.6355025	total: 49.8s	remaining: 1m 19s
386:	learn: 0.6350523	total: 50s	remaining: 1m 19s
387:	learn: 0.6348004	total: 50.1s	remaining: 1m 19s
388:	learn: 0.6344975	total: 50.2s	remaining: 1m 18s
389:	learn: 0.6342227	total: 50.4s	remaining: 1m 18s
390:	learn: 0.6339415	total: 50.5s	remaining: 1m 18s
391:	learn: 0.6334379	total: 50.6s	remaining: 1m 18s
392:	learn: 0.6331893	total: 50.8s	remaining: 1m 18s
393:	learn: 0.6326834	total: 50.9s	remaining: 1m 18s
394:	learn: 0.6319040	total: 51s	remaining: 1m 18s
395:	learn: 0.6316324	total: 51.1s	remaining: 1m 18s
396:	learn: 0.6313356	total: 51.3s	remaining: 1m 17s
397:	learn: 0.6303765	total: 51.4s	remaining: 1m 17s
398:	learn: 0.6300160	total: 51.5s	remaining: 1m 1

538:	learn: 0.5893194	total: 1m 9s	remaining: 59.3s
539:	learn: 0.5891688	total: 1m 9s	remaining: 59.2s
540:	learn: 0.5889028	total: 1m 9s	remaining: 59.1s
541:	learn: 0.5886790	total: 1m 9s	remaining: 58.9s
542:	learn: 0.5884123	total: 1m 9s	remaining: 58.8s
543:	learn: 0.5882058	total: 1m 9s	remaining: 58.7s
544:	learn: 0.5879628	total: 1m 10s	remaining: 58.5s
545:	learn: 0.5877058	total: 1m 10s	remaining: 58.4s
546:	learn: 0.5873852	total: 1m 10s	remaining: 58.3s
547:	learn: 0.5872025	total: 1m 10s	remaining: 58.2s
548:	learn: 0.5870457	total: 1m 10s	remaining: 58s
549:	learn: 0.5868181	total: 1m 10s	remaining: 57.9s
550:	learn: 0.5866376	total: 1m 10s	remaining: 57.8s
551:	learn: 0.5862261	total: 1m 11s	remaining: 57.6s
552:	learn: 0.5860991	total: 1m 11s	remaining: 57.5s
553:	learn: 0.5858635	total: 1m 11s	remaining: 57.4s
554:	learn: 0.5856540	total: 1m 11s	remaining: 57.2s
555:	learn: 0.5854460	total: 1m 11s	remaining: 57.1s
556:	learn: 0.5851597	total: 1m 11s	remaining: 57s
557

694:	learn: 0.5520817	total: 1m 29s	remaining: 39.4s
695:	learn: 0.5518906	total: 1m 29s	remaining: 39.2s
696:	learn: 0.5517235	total: 1m 29s	remaining: 39.1s
697:	learn: 0.5513648	total: 1m 30s	remaining: 39s
698:	learn: 0.5512007	total: 1m 30s	remaining: 38.8s
699:	learn: 0.5510383	total: 1m 30s	remaining: 38.7s
700:	learn: 0.5507446	total: 1m 30s	remaining: 38.6s
701:	learn: 0.5506494	total: 1m 30s	remaining: 38.5s
702:	learn: 0.5503334	total: 1m 30s	remaining: 38.3s
703:	learn: 0.5502268	total: 1m 30s	remaining: 38.2s
704:	learn: 0.5499697	total: 1m 30s	remaining: 38.1s
705:	learn: 0.5497313	total: 1m 31s	remaining: 37.9s
706:	learn: 0.5495027	total: 1m 31s	remaining: 37.8s
707:	learn: 0.5493158	total: 1m 31s	remaining: 37.7s
708:	learn: 0.5491386	total: 1m 31s	remaining: 37.5s
709:	learn: 0.5488892	total: 1m 31s	remaining: 37.4s
710:	learn: 0.5486871	total: 1m 31s	remaining: 37.3s
711:	learn: 0.5484264	total: 1m 31s	remaining: 37.2s
712:	learn: 0.5483204	total: 1m 31s	remaining: 3

850:	learn: 0.5223644	total: 1m 49s	remaining: 19.2s
851:	learn: 0.5222322	total: 1m 49s	remaining: 19.1s
852:	learn: 0.5221211	total: 1m 50s	remaining: 19s
853:	learn: 0.5216157	total: 1m 50s	remaining: 18.8s
854:	learn: 0.5214087	total: 1m 50s	remaining: 18.7s
855:	learn: 0.5212259	total: 1m 50s	remaining: 18.6s
856:	learn: 0.5210892	total: 1m 50s	remaining: 18.4s
857:	learn: 0.5209675	total: 1m 50s	remaining: 18.3s
858:	learn: 0.5208106	total: 1m 50s	remaining: 18.2s
859:	learn: 0.5207053	total: 1m 50s	remaining: 18.1s
860:	learn: 0.5201125	total: 1m 51s	remaining: 17.9s
861:	learn: 0.5198788	total: 1m 51s	remaining: 17.8s
862:	learn: 0.5196050	total: 1m 51s	remaining: 17.7s
863:	learn: 0.5194661	total: 1m 51s	remaining: 17.5s
864:	learn: 0.5193344	total: 1m 51s	remaining: 17.4s
865:	learn: 0.5192266	total: 1m 51s	remaining: 17.3s
866:	learn: 0.5189998	total: 1m 51s	remaining: 17.2s
867:	learn: 0.5188624	total: 1m 51s	remaining: 17s
868:	learn: 0.5187606	total: 1m 52s	remaining: 16.

ValueError: multi:softmax doesn't support `predict_proba`.  Switch to `multi:softproba` instead

In [ ]:
sclf.fit(features_train, labels_train)

0:	learn: 1.0714499	total: 145ms	remaining: 2m 24s
1:	learn: 1.0476183	total: 323ms	remaining: 2m 41s
2:	learn: 1.0309258	total: 514ms	remaining: 2m 50s
3:	learn: 1.0150183	total: 716ms	remaining: 2m 58s
4:	learn: 0.9996126	total: 888ms	remaining: 2m 56s
5:	learn: 0.9878205	total: 1.04s	remaining: 2m 52s
6:	learn: 0.9791843	total: 1.18s	remaining: 2m 47s
7:	learn: 0.9728351	total: 1.32s	remaining: 2m 43s
8:	learn: 0.9657334	total: 1.46s	remaining: 2m 40s
9:	learn: 0.9606416	total: 1.6s	remaining: 2m 38s
10:	learn: 0.9525555	total: 1.74s	remaining: 2m 36s
11:	learn: 0.9445354	total: 1.89s	remaining: 2m 35s
12:	learn: 0.9361199	total: 2.02s	remaining: 2m 33s
13:	learn: 0.9318000	total: 2.16s	remaining: 2m 32s
14:	learn: 0.9260376	total: 2.31s	remaining: 2m 31s
15:	learn: 0.9221229	total: 2.48s	remaining: 2m 32s
16:	learn: 0.9180523	total: 2.67s	remaining: 2m 34s
17:	learn: 0.9152151	total: 2.88s	remaining: 2m 36s
18:	learn: 0.9105569	total: 3.05s	remaining: 2m 37s
19:	learn: 0.9084039	to

158:	learn: 0.7495991	total: 25s	remaining: 2m 12s
159:	learn: 0.7491310	total: 25.2s	remaining: 2m 12s
160:	learn: 0.7479610	total: 25.3s	remaining: 2m 11s
161:	learn: 0.7477024	total: 25.4s	remaining: 2m 11s
162:	learn: 0.7472141	total: 25.6s	remaining: 2m 11s
163:	learn: 0.7462724	total: 25.7s	remaining: 2m 11s
164:	learn: 0.7451703	total: 25.9s	remaining: 2m 10s
165:	learn: 0.7448285	total: 26s	remaining: 2m 10s
166:	learn: 0.7441831	total: 26.1s	remaining: 2m 10s
167:	learn: 0.7432144	total: 26.3s	remaining: 2m 10s
168:	learn: 0.7417973	total: 26.4s	remaining: 2m 9s
169:	learn: 0.7412232	total: 26.5s	remaining: 2m 9s
170:	learn: 0.7406870	total: 26.7s	remaining: 2m 9s
171:	learn: 0.7396632	total: 26.8s	remaining: 2m 9s
172:	learn: 0.7392564	total: 27s	remaining: 2m 8s
173:	learn: 0.7388756	total: 27.1s	remaining: 2m 8s
174:	learn: 0.7384412	total: 27.2s	remaining: 2m 8s
175:	learn: 0.7378962	total: 27.4s	remaining: 2m 8s
176:	learn: 0.7373233	total: 27.5s	remaining: 2m 7s
177:	lea

316:	learn: 0.6677363	total: 47s	remaining: 1m 41s
317:	learn: 0.6674245	total: 47.1s	remaining: 1m 41s
318:	learn: 0.6672369	total: 47.3s	remaining: 1m 40s
319:	learn: 0.6669273	total: 47.4s	remaining: 1m 40s
320:	learn: 0.6666393	total: 47.5s	remaining: 1m 40s
321:	learn: 0.6660950	total: 47.7s	remaining: 1m 40s
322:	learn: 0.6657765	total: 47.8s	remaining: 1m 40s
323:	learn: 0.6653059	total: 47.9s	remaining: 1m 40s
324:	learn: 0.6648797	total: 48.1s	remaining: 1m 39s
325:	learn: 0.6645491	total: 48.2s	remaining: 1m 39s
326:	learn: 0.6638085	total: 48.4s	remaining: 1m 39s
327:	learn: 0.6635688	total: 48.5s	remaining: 1m 39s
328:	learn: 0.6632401	total: 48.6s	remaining: 1m 39s
329:	learn: 0.6629182	total: 48.8s	remaining: 1m 39s
330:	learn: 0.6626730	total: 48.9s	remaining: 1m 38s
331:	learn: 0.6622853	total: 49.1s	remaining: 1m 38s
332:	learn: 0.6618466	total: 49.2s	remaining: 1m 38s
333:	learn: 0.6613884	total: 49.3s	remaining: 1m 38s
334:	learn: 0.6610290	total: 49.5s	remaining: 1m

472:	learn: 0.6179498	total: 1m 9s	remaining: 1m 17s
473:	learn: 0.6176789	total: 1m 10s	remaining: 1m 17s
474:	learn: 0.6174049	total: 1m 10s	remaining: 1m 17s
475:	learn: 0.6172067	total: 1m 10s	remaining: 1m 17s
476:	learn: 0.6170558	total: 1m 10s	remaining: 1m 17s
477:	learn: 0.6168794	total: 1m 10s	remaining: 1m 17s
478:	learn: 0.6165090	total: 1m 10s	remaining: 1m 17s
479:	learn: 0.6161719	total: 1m 10s	remaining: 1m 16s
480:	learn: 0.6158687	total: 1m 11s	remaining: 1m 16s
481:	learn: 0.6153429	total: 1m 11s	remaining: 1m 16s
482:	learn: 0.6151504	total: 1m 11s	remaining: 1m 16s
483:	learn: 0.6149672	total: 1m 11s	remaining: 1m 16s
484:	learn: 0.6148127	total: 1m 11s	remaining: 1m 16s
485:	learn: 0.6146287	total: 1m 11s	remaining: 1m 15s
486:	learn: 0.6144167	total: 1m 11s	remaining: 1m 15s
487:	learn: 0.6141441	total: 1m 12s	remaining: 1m 15s
488:	learn: 0.6137197	total: 1m 12s	remaining: 1m 15s
489:	learn: 0.6134201	total: 1m 12s	remaining: 1m 15s
490:	learn: 0.6132633	total: 

628:	learn: 0.5803855	total: 1m 32s	remaining: 54.3s
629:	learn: 0.5801313	total: 1m 32s	remaining: 54.2s
630:	learn: 0.5794138	total: 1m 32s	remaining: 54s
631:	learn: 0.5792290	total: 1m 32s	remaining: 53.9s
632:	learn: 0.5790330	total: 1m 32s	remaining: 53.7s
633:	learn: 0.5788426	total: 1m 32s	remaining: 53.6s
634:	learn: 0.5786289	total: 1m 32s	remaining: 53.5s
635:	learn: 0.5785035	total: 1m 33s	remaining: 53.3s
636:	learn: 0.5781549	total: 1m 33s	remaining: 53.2s
637:	learn: 0.5779917	total: 1m 33s	remaining: 53s
638:	learn: 0.5777566	total: 1m 33s	remaining: 52.9s
639:	learn: 0.5776174	total: 1m 33s	remaining: 52.7s
640:	learn: 0.5773570	total: 1m 33s	remaining: 52.6s
641:	learn: 0.5772035	total: 1m 33s	remaining: 52.4s
642:	learn: 0.5770335	total: 1m 34s	remaining: 52.3s
643:	learn: 0.5768368	total: 1m 34s	remaining: 52.1s
644:	learn: 0.5765155	total: 1m 34s	remaining: 52s
645:	learn: 0.5763196	total: 1m 34s	remaining: 51.8s
646:	learn: 0.5761060	total: 1m 34s	remaining: 51.7s

...

942:	learn: 0.5184012	total: 2m 23s	remaining: 8.69s
943:	learn: 0.5182697	total: 2m 23s	remaining: 8.53s
944:	learn: 0.5181548	total: 2m 24s	remaining: 8.38s
945:	learn: 0.5180041	total: 2m 24s	remaining: 8.23s
946:	learn: 0.5177062	total: 2m 24s	remaining: 8.07s
947:	learn: 0.5175819	total: 2m 24s	remaining: 7.92s
948:	learn: 0.5174900	total: 2m 24s	remaining: 7.77s
949:	learn: 0.5174046	total: 2m 24s	remaining: 7.62s
950:	learn: 0.5172928	total: 2m 24s	remaining: 7.46s
951:	learn: 0.5171349	total: 2m 25s	remaining: 7.31s
952:	learn: 0.5169577	total: 2m 25s	remaining: 7.16s
953:	learn: 0.5167779	total: 2m 25s	remaining: 7s
954:	learn: 0.5166004	total: 2m 25s	remaining: 6.85s
955:	learn: 0.5162583	total: 2m 25s	remaining: 6.7s
956:	learn: 0.5161293	total: 2m 25s	remaining: 6.55s
957:	learn: 0.5159342	total: 2m 25s	remaining: 6.4s
958:	learn: 0.5157701	total: 2m 26s	remaining: 6.24s
959:	learn: 0.5157022	total: 2m 26s	remaining: 6.09s
960:	learn: 0.5155753	total: 2m 26s	remaining: 5.94

102:	learn: 0.8031568	total: 13.4s	remaining: 1m 56s
103:	learn: 0.8022075	total: 13.5s	remaining: 1m 56s
104:	learn: 0.8003325	total: 13.7s	remaining: 1m 56s
105:	learn: 0.7976495	total: 13.8s	remaining: 1m 56s
106:	learn: 0.7972127	total: 13.9s	remaining: 1m 56s
107:	learn: 0.7962565	total: 14s	remaining: 1m 55s
108:	learn: 0.7956902	total: 14.2s	remaining: 1m 55s
109:	learn: 0.7946016	total: 14.3s	remaining: 1m 55s
110:	learn: 0.7933716	total: 14.4s	remaining: 1m 55s
111:	learn: 0.7922600	total: 14.6s	remaining: 1m 55s
112:	learn: 0.7909285	total: 14.7s	remaining: 1m 55s
113:	learn: 0.7900179	total: 14.8s	remaining: 1m 55s
114:	learn: 0.7891650	total: 14.9s	remaining: 1m 54s
115:	learn: 0.7876197	total: 15.1s	remaining: 1m 54s
116:	learn: 0.7864260	total: 15.2s	remaining: 1m 54s
117:	learn: 0.7854527	total: 15.3s	remaining: 1m 54s
118:	learn: 0.7845706	total: 15.5s	remaining: 1m 54s
119:	learn: 0.7831706	total: 15.6s	remaining: 1m 54s
120:	learn: 0.7824695	total: 15.7s	remaining: 1m

258:	learn: 0.6895053	total: 33.6s	remaining: 1m 36s
259:	learn: 0.6891088	total: 33.7s	remaining: 1m 35s
260:	learn: 0.6888241	total: 33.8s	remaining: 1m 35s
261:	learn: 0.6884045	total: 34s	remaining: 1m 35s
262:	learn: 0.6880981	total: 34.1s	remaining: 1m 35s
263:	learn: 0.6878218	total: 34.2s	remaining: 1m 35s
264:	learn: 0.6874171	total: 34.4s	remaining: 1m 35s
265:	learn: 0.6870730	total: 34.5s	remaining: 1m 35s
266:	learn: 0.6867843	total: 34.6s	remaining: 1m 35s
267:	learn: 0.6864782	total: 34.7s	remaining: 1m 34s
268:	learn: 0.6859623	total: 34.9s	remaining: 1m 34s
269:	learn: 0.6856471	total: 35s	remaining: 1m 34s
270:	learn: 0.6853936	total: 35.1s	remaining: 1m 34s
271:	learn: 0.6850289	total: 35.3s	remaining: 1m 34s
272:	learn: 0.6845674	total: 35.4s	remaining: 1m 34s
273:	learn: 0.6840817	total: 35.6s	remaining: 1m 34s
274:	learn: 0.6837458	total: 35.7s	remaining: 1m 34s
275:	learn: 0.6831849	total: 35.8s	remaining: 1m 34s
276:	learn: 0.6828090	total: 36s	remaining: 1m 33s

414:	learn: 0.6285178	total: 54.2s	remaining: 1m 16s
415:	learn: 0.6282058	total: 54.4s	remaining: 1m 16s
416:	learn: 0.6278477	total: 54.5s	remaining: 1m 16s
417:	learn: 0.6276070	total: 54.6s	remaining: 1m 16s
418:	learn: 0.6273856	total: 54.8s	remaining: 1m 15s
419:	learn: 0.6271745	total: 54.9s	remaining: 1m 15s
420:	learn: 0.6267713	total: 55s	remaining: 1m 15s
421:	learn: 0.6264973	total: 55.1s	remaining: 1m 15s
422:	learn: 0.6263941	total: 55.3s	remaining: 1m 15s
423:	learn: 0.6261413	total: 55.4s	remaining: 1m 15s
424:	learn: 0.6259252	total: 55.5s	remaining: 1m 15s
425:	learn: 0.6255895	total: 55.6s	remaining: 1m 14s
426:	learn: 0.6253442	total: 55.8s	remaining: 1m 14s
427:	learn: 0.6250712	total: 55.9s	remaining: 1m 14s
428:	learn: 0.6247899	total: 56s	remaining: 1m 14s
429:	learn: 0.6243251	total: 56.2s	remaining: 1m 14s
430:	learn: 0.6239967	total: 56.3s	remaining: 1m 14s
431:	learn: 0.6234051	total: 56.4s	remaining: 1m 14s
432:	learn: 0.6231073	total: 56.6s	remaining: 1m 1

572:	learn: 0.5858593	total: 1m 15s	remaining: 55.9s
573:	learn: 0.5855755	total: 1m 15s	remaining: 55.8s
574:	learn: 0.5853359	total: 1m 15s	remaining: 55.6s
575:	learn: 0.5850814	total: 1m 15s	remaining: 55.5s
576:	learn: 0.5848218	total: 1m 15s	remaining: 55.4s
577:	learn: 0.5846515	total: 1m 15s	remaining: 55.2s
578:	learn: 0.5844320	total: 1m 15s	remaining: 55.1s
579:	learn: 0.5843247	total: 1m 15s	remaining: 55s
580:	learn: 0.5840142	total: 1m 16s	remaining: 54.8s
581:	learn: 0.5838050	total: 1m 16s	remaining: 54.7s
582:	learn: 0.5835989	total: 1m 16s	remaining: 54.6s
583:	learn: 0.5833476	total: 1m 16s	remaining: 54.4s
584:	learn: 0.5831179	total: 1m 16s	remaining: 54.3s
585:	learn: 0.5829732	total: 1m 16s	remaining: 54.2s
586:	learn: 0.5827810	total: 1m 16s	remaining: 54s
587:	learn: 0.5826193	total: 1m 16s	remaining: 53.9s
588:	learn: 0.5823723	total: 1m 17s	remaining: 53.8s
589:	learn: 0.5820956	total: 1m 17s	remaining: 53.6s
590:	learn: 0.5818094	total: 1m 17s	remaining: 53.